# 데이터 전처리 이해와 실무

## `데이터 변환: 구간화, 정규화`

### 구간화, 정규화(강의 교안)
1. 변수 변환 방안
   - 구간화: 연속형 데이터의 범주화
   - 정규화: 연속형 데이터들의 관측 간격 조정(스케일링)

## 0. 라이브러리 및 데이터 로딩

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 데이터 로딩
# 데이터 로딩 및 개요 확인
click_data = pd.read_csv("./data/article_click.csv")
click_data.head(15)

,category,Journal,article_id,num_click
0,스포츠,D일보,ud0Rt3iBUX,3719
1,사회,B일보,v7oCsXJjMn,1271
2,경제,B일보,bvrGMNy2VL,1929
3,경제,B일보,4vNnU38N3m,5728
4,사설,B일보,NWZpabeNb7,4077
5,부동산,D일보,3wEfH9cXjy,642
6,스포츠,B일보,5gpjKc84PS,1887
7,부동산,D일보,Ccvi807Gvv,914
8,스포츠,B일보,QstXJUr49N,938
9,정치,B일보,0ZvQknTOKK,1279


In [3]:
# 데이터 개요 파악
click_data.info()

# 데이터 copy
click_copy = click_data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17734 entries, 0 to 17733
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    17734 non-null  object
 1   Journal     17734 non-null  object
 2   article_id  17734 non-null  object
 3   num_click   17734 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 554.3+ KB


## 1. 구간화(Binning)
#### 구간화 방안
1. 지정 길이 기반 구간: 데이터 범위를 사용자의 기준대로 정의하고 구간화
2. 분포 기반 구간: 각 데이터가 모든 구간 내 동일한 개수로 구분되도록 구간화

### 1-1. `지정 길이 기반 구간`

In [4]:
click_copy.describe() # 기술통계량 조회

,num_click
count,17734.000000
mean,1348.052611
std,1185.450912
min,443.000000
25%,717.000000
50%,986.000000
75%,1521.000000
max,25661.000000


- 여기에서 목적을 한 가지 가정해보자.
- 만약 기사의 선호도를 파악하고자 할 때에는 단순 클릭 수를 선호도 정도로 볼 수 있을까 하는 것이다.
- 만약 기사의 클릭수가 20회인 기사와 10회인 기사를 비교했을 때, 20회 클릭된 기사가 2배 더 흥미있는 기사로 정의할 수 있는가?
  - 없음! 기사의 카테고리, 저널 등 클릭을 유도하는 여러 요소가 있기 때문
- 따라서, 구간화를 통해 새로운 의미를 지닌 변수로 변환이 필요하며 목적 설정이 중요함
- 클릭 횟수를 기준으로 범주로 변환하여 각 범주별 의미 설정 가능(도메인 지식 검토 필요)

In [5]:
# 지정 길이 기반 구간화: cut 함수
# 클릭 수 기반으로 기사 관심도 정의 기반 범주형 변환(3개 구간 설정) : 3가지 범주로 선호도를 구분한다고 해보자. 

# cut 함수 활용 기반 구간 설정 (사용자 기준)
bins = [0, 100, 500, np.max(click_copy['num_click'])] # 구간별 범위를 사전에 지정해야 # 100, 500, 최대
names = ['low', 'medium', 'high'] # 각 범주의 이름 제시 

click_copy['pref'] = pd.cut(click_copy['num_click'], bins, labels=names) 
# .cut(구간화 진행 변수, 사전에 설정한 구간 범위, labels=레이블이름

In [6]:
# 구간 결과 확인
click_copy['pref'].value_counts()

pref
high      17413
medium      321
low           0
Name: count, dtype: int64

- 클릭횟수의 범위를 임의로 구분하였기 때문에 구간마다 다른 데이터 개수를 보유하게 되었다. 

In [7]:
click_copy.head(15) # 클릭 횟수를 기반으로 한 구간 칼럼이 추가됨

,category,Journal,article_id,num_click,pref
0,스포츠,D일보,ud0Rt3iBUX,3719,high
1,사회,B일보,v7oCsXJjMn,1271,high
2,경제,B일보,bvrGMNy2VL,1929,high
3,경제,B일보,4vNnU38N3m,5728,high
4,사설,B일보,NWZpabeNb7,4077,high
5,부동산,D일보,3wEfH9cXjy,642,high
6,스포츠,B일보,5gpjKc84PS,1887,high
7,부동산,D일보,Ccvi807Gvv,914,high
8,스포츠,B일보,QstXJUr49N,938,high
9,정치,B일보,0ZvQknTOKK,1279,high


### 1-2. `분포 기반 구간`

In [8]:
# qcut의 경우 동일한 관측치가 배치되도록 구간을 분할
# 따라서 구간수로 나누게 됨
# 구간수 = n
n = 3 # 사전에 나눌 개수로 넣는다. 
click_copy['pref_qcut'] = pd.qcut(click_copy['num_click'], n, labels=names)
print(click_copy['pref_qcut'].value_counts())

pref_qcut
low       5915
high      5912
medium    5907
Name: count, dtype: int64


- 데이터가 정확히 3으로 나누어지는 데이터는 아니었기에, high의 범주에서는 거의 유사하지만 조금 차이가 난다. 

> 만일 qcut 실행 시 주로 발생하는 오류
- 데이터의 극단치의 Skewed(치우친) 한 경우 발생 -> 입력한 분포 개수에 따라 구간화가 실행되지 않을 수 있
- 데이터의 각 구간은 동일할 수 없기 때문
  - [1,1,1,1,1,3,8,10]이라는 구간은 생성 불가능
- 그러한 경우에는 분포 기반 구간화(qcut)보다 사용자 기준 기반 범주화 혹은 구간 개수 변화 고려해야!

## 2. 정규화 
> 기계학습 알고리즘의 각 변수별 영향력의 차이를 조정할 필요 존재함
- 각 변수별 범위가 다를 경우 학습 안정성이 떨어질 수 있으므로 간격 조정 필요

1. 최대-최소 정규화: 변수 안의 값을 [0,1] 구간의 값을 갖도록 구성하고 데이터 군 내에서 특정 데이터가 가지는 위치를 보고자 할 떄 사용
2. z-score 정규화: 특정 데이터가 평균을 기준으로 얼마나 떨어져 있는지 파악하고 측정 스케일이 다른 경우의 데이터를 비교하는데 사용

### 2-1. `최대-최소 정규화`

In [9]:
# 최대-최소 정규화(Min-Max Scaling)
# 변수 값을 0에서 1 사이의 값으로 변환
# 함수는 sklearn 라이브러리에 쉽게 구현되어 있음

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() # scaler 정의하

# 데이터 copy
click_copy = click_data.copy()

# 변환
click_copy['minmax_values'] = scaler.fit_transform(click_copy[['num_click']]) 
# 'minmax_values'라고 별도 컬럼 지정해보자 # .fit_transform : 학습피팅과 변환을 한꺼번에 해봄
click_copy.head(15)

,category,Journal,article_id,num_click,minmax_values
0,스포츠,D일보,ud0Rt3iBUX,3719,0.129907
1,사회,B일보,v7oCsXJjMn,1271,0.032834
2,경제,B일보,bvrGMNy2VL,1929,0.058926
3,경제,B일보,4vNnU38N3m,5728,0.209573
4,사설,B일보,NWZpabeNb7,4077,0.144103
5,부동산,D일보,3wEfH9cXjy,642,0.007891
6,스포츠,B일보,5gpjKc84PS,1887,0.057261
7,부동산,D일보,Ccvi807Gvv,914,0.018677
8,스포츠,B일보,QstXJUr49N,938,0.019629
9,정치,B일보,0ZvQknTOKK,1279,0.033151


- 각 기사 클릭수별 관측 범위 내에서 데이터가 가지는 위치를 표현한 최대-최소 정규화 값이 minmax_values 컬럼에 생성되었음을 확인 가능!

In [10]:
# 정말 minmax_values가 0, 1 사이의 값으로 변화했는지 보자. 
# 데이터 요약
click_copy.describe()

,num_click,minmax_values
count,17734.000000,17734.000000
mean,1348.052611,0.035889
std,1185.450912,0.047008
min,443.000000,0.000000
25%,717.000000,0.010865
50%,986.000000,0.021532
75%,1521.000000,0.042747
max,25661.000000,1.000000


### 2-. `Z-score 정규화(Z-Score Normalization, 표준화)`

In [12]:
# 변수를 평균이 0이고 표준편차가 1인 표준정규분포로 변환
# 함수는 sklearn 라이브러리에 쉽게 구현되어 있음

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

# 변환
click_copy['std_values'] = std_scaler.fit_transform(click_copy[['num_click']])
click_copy.head(15)

,category,Journal,article_id,num_click,minmax_values,std_values
0,스포츠,D일보,ud0Rt3iBUX,3719,0.129907,2.000095
1,사회,B일보,v7oCsXJjMn,1271,0.032834,-0.065000
2,경제,B일보,bvrGMNy2VL,1929,0.058926,0.490078
3,경제,B일보,4vNnU38N3m,5728,0.209573,3.694856
4,사설,B일보,NWZpabeNb7,4077,0.144103,2.302098
5,부동산,D일보,3wEfH9cXjy,642,0.007891,-0.595615
6,스포츠,B일보,5gpjKc84PS,1887,0.057261,0.454648
7,부동산,D일보,Ccvi807Gvv,914,0.018677,-0.366160
8,스포츠,B일보,QstXJUr49N,938,0.019629,-0.345914
9,정치,B일보,0ZvQknTOKK,1279,0.033151,-0.058252


In [13]:
# 데이터 요약
click_copy.describe()
# 이상치 제거 후 수치형 데이터를 변환하여 모델링 적용 필요

,num_click,minmax_values,std_values
count,17734.000000,17734.000000,1.773400e+04
mean,1348.052611,0.035889,-1.903168e-17
std,1185.450912,0.047008,1.000028e+00
min,443.000000,0.000000,-7.634885e-01
25%,717.000000,0.010865,-5.323463e-01
50%,986.000000,0.021532,-3.054220e-01
75%,1521.000000,0.042747,1.458958e-01
max,25661.000000,1.000000,2.051003e+01


> 어느 방안이 더 나은 것인가?
- 상황에 따라 달라지며, 정규화 수행과 수행하지 않은 결과의 차이가 큰 것이 중요한 포인트임
- 연속형 데이터를 기계학습 모델링에 적용하기 위해서 변수 간의 상대적 크기 차이를 제거할 필요가 반드시 존재함
- ex)
  - 대표적 비지도학습인 군집분석 중 Kmeans는 수치형 변수로 적용하는 알고리즘
  - 변수 간 수치적 distance(거리)를 연산하여 유사한 집단으로 묶는 알고리즘
  - 변수가 각자 다른 범위를 가진 경우 유사 집단 구성 시, 변수 간 영향력의 차이가 반영되어 올바르지 못한 결과 도출 가능성 존재